In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import math
import matplotlib.pyplot as plt
import warnings
import numpy as np
import sqlite3 as sql
from mlxtend.frequent_patterns import apriori, association_rules
warnings.simplefilter('ignore')

In [2]:
conn = sql.connect('..\Databases\GreatOutdoors\go_sales.sqlite')
query = "SELECT * FROM product"
query2 = "SELECT * FROM order_details"
df = pd.read_sql(query,conn)
df2 = pd.read_sql(query2,conn)

df = df.merge(df2, on='PRODUCT_NUMBER')

df

,PRODUCT_NUMBER,INTRODUCTION_DATE,PRODUCT_TYPE_CODE,PRODUCTION_COST,MARGIN,PRODUCT_IMAGE,LANGUAGE,PRODUCT_NAME,DESCRIPTION,TRIAL888,ORDER_DETAIL_CODE,ORDER_NUMBER,QUANTITY,UNIT_COST,UNIT_PRICE,UNIT_SALE_PRICE,TRIAL879
0,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,100085,8474,40,4.01,6.03,6.03,T
1,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,100086,8470,40,4.01,6.03,6.03,T
2,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,100087,8464,48,4.01,6.03,6.03,T
3,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,100088,9254,66,4.01,6.03,6.03,T
4,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,100089,8472,88,4.01,6.03,6.03,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43058,99,15-2-2011,17,2,.6,P99OP4FA17.jpg,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",T,98478,8210,30,1.97,4.94,4.94,T
43059,99,15-2-2011,17,2,.6,P99OP4FA17.jpg,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",T,99566,8246,20,2,5,3.95,T
43060,99,15-2-2011,17,2,.6,P99OP4FA17.jpg,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",T,99567,8239,18,2,5,5,T
43061,99,15-2-2011,17,2,.6,P99OP4FA17.jpg,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",T,99568,8270,26,2,5,5,T


In [3]:
df = df.loc[:,['PRODUCT_NUMBER','PRODUCT_TYPE_CODE','ORDER_NUMBER','UNIT_PRICE']]
df['PRODUCT_NUMBER'] = pd.to_numeric(df['PRODUCT_NUMBER'], errors='coerce')
df['PRODUCT_TYPE_CODE'] = pd.to_numeric(df['PRODUCT_TYPE_CODE'], errors='coerce')
df['ORDER_NUMBER'] = pd.to_numeric(df['ORDER_NUMBER'], errors='coerce')
df.fillna(method='ffill', inplace=True)  # Fill missing values
df['UNIT_PRICE'] = pd.to_numeric(df['UNIT_PRICE'], errors='coerce')
df

,PRODUCT_NUMBER,PRODUCT_TYPE_CODE,ORDER_NUMBER,UNIT_PRICE
0,1,1,8474,6.03
1,1,1,8470,6.03
2,1,1,8464,6.03
3,1,1,9254,6.03
4,1,1,8472,6.03
...,...,...,...,...
43058,99,17,8210,4.94
43059,99,17,8246,5.00
43060,99,17,8239,5.00
43061,99,17,8270,5.00


In [5]:

#'ORDER_NUMBER' is de transaction identifier
df_grouped = df.groupby('ORDER_NUMBER')['PRODUCT_NUMBER'].apply(list).reset_index()

# Convert product numbers into item presence within each transaction (one-hot encoding)
transactions = df_grouped['PRODUCT_NUMBER'].tolist()
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
transactions_df = pd.DataFrame(te_ary, columns=te.columns_)

# Define minimum support and confidence thresholds
min_support = 0.02
min_confidence = 0.5

# Apply Apriori algorithm
frequent_itemsets = apriori(transactions_df, min_support=min_support, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)

pd.set_option('display.max_rows', None)
# Print frequent itemsets and association rules
print("Frequent Itemsets:")
print(frequent_itemsets)
print("\nAssociation Rules:")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].to_string(index=False))

Frequent Itemsets:
      support    itemsets
0    0.086381         (1)
1    0.078172         (2)
2    0.084888         (3)
3    0.076493         (4)
4    0.075373         (5)
5    0.055970         (6)
6    0.067724         (7)
7    0.083582         (8)
8    0.085821         (9)
9    0.081716        (10)
10   0.066978        (11)
11   0.086007        (12)
12   0.073507        (13)
13   0.083022        (14)
14   0.069403        (15)
15   0.058396        (16)
16   0.099067        (17)
17   0.084701        (18)
18   0.087313        (19)
19   0.071642        (20)
20   0.064739        (21)
21   0.054851        (22)
22   0.054104        (23)
23   0.081157        (24)
24   0.060075        (25)
25   0.080784        (26)
26   0.083209        (27)
27   0.084142        (28)
28   0.106903        (29)
29   0.063433        (30)
30   0.068470        (31)
31   0.097948        (32)
32   0.088806        (33)
33   0.081157        (34)
34   0.068097        (35)
35   0.077052        (36)
36   0.046642      

support: (percentage) zoveel procent van de transacties bevat allebei de producten
confidence: gebruikt om voorspellingen te doen op basis van transacties met meerdere producten. Bijvoorbeeld:
Support van product {1,2,3} is 0.03 (3%)
Support van product {1,2} is 0.05 (5%)

dan betekent is de confidence 0.03/0.05 = 0.6 of 60%.
Dit betekent dat in 60% van de transacties waarin product 1 en 2 worden gekocht, product 3 ook gekocht wordt.